### Import and Config

In [1]:
import os

from model_setup import Model_Setup
from PoseDataGenerator import PoseDataGenerator
from Plot3D import Landmark_list,plot_landmarks
from Train import Train

PATH_VIDEO = 'videos/1.mp4'
RESULT_CSV = 'results/coordinates.csv'

config = Model_Setup()


### Generate training data from Video Clip

In [2]:
pose_data = PoseDataGenerator(config)
if os.path.isfile(RESULT_CSV) == False:
    pose_data.generate_pose(PATH_VIDEO,RESULT_CSV)

### Testing 3DPlot Pose Estimation

In [3]:
landmark_list = Landmark_list(list_size = config.NUM_COORDS)
df = landmark_list.load_csv(RESULT_CSV)

In [4]:
'''
# Plot every frame
for i in range(len(df)):
    landmark_list = Landmark_list(list_size = 33)
    df_temp = df.iloc[i,:]
    x = df_temp[df.columns.str.startswith('x')].to_numpy().flatten()
    y = df_temp[df.columns.str.startswith('y')].to_numpy().flatten()
    z = df_temp[df.columns.str.startswith('z')].to_numpy().flatten()
    visibility = df_temp[df.columns.str.startswith('v')].to_numpy().flatten()
    landmark_list.load_xyz(x,y,z,visibility)
    plot_landmarks(landmark_list, pose_data.mp_pose.POSE_CONNECTIONS)
'''

"\n# Plot every frame\nfor i in range(len(df)):\n    landmark_list = Landmark_list(list_size = 33)\n    df_temp = df.iloc[i,:]\n    x = df_temp[df.columns.str.startswith('x')].to_numpy().flatten()\n    y = df_temp[df.columns.str.startswith('y')].to_numpy().flatten()\n    z = df_temp[df.columns.str.startswith('z')].to_numpy().flatten()\n    visibility = df_temp[df.columns.str.startswith('v')].to_numpy().flatten()\n    landmark_list.load_xyz(x,y,z,visibility)\n    plot_landmarks(landmark_list, pose_data.mp_pose.POSE_CONNECTIONS)\n"

###  Train 


In [ ]:
train_obj = Train(config)
train_obj.fit(df)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 30, 400)           480000    
_________________________________________________________________
dense (Dense)                (None, 30, 100)           40100     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               301200    
_________________________________________________________________
dense_1 (Dense)              (None, 20)                6020      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 827,761
Trainable params: 827,761
Non-trainable params: 0
__________________________________________________

In [ ]:
train_obj.plot_performance()